# Serving a Session-based Recommendation model with Torch Backend

* following [this tutrial](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/examples/getting-started-session-based/03-serving-session-based-model-torch-backend.ipynb)

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf
import glob
import torch 

from transformers4rec import torch as tr
from merlin.io import Dataset

from merlin.core.dispatch import make_df  # noqa
from merlin.systems.dag import Ensemble  # noqa
from merlin.systems.dag.ops.pytorch import PredictPyTorch  # noqa

In [7]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'
REGION = "us-central1"

# VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com'
VERTEX_SA = 'jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")
print(f"REGION: {REGION}")
print(f"VERTEX_SA: {VERTEX_SA}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1
REGION: us-central1
VERTEX_SA: jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com


In [8]:
# INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data")
# OUTPUT_DIR = os.environ.get("OUTPUT_DIR", f"{INPUT_DATA_DIR}/sessions_by_day")
# model_path= os.environ.get("model_path", f"{INPUT_DATA_DIR}/saved_model")

REPO_WORKSPACE = 'workspace'
DATA_DIR = 'data'
INPUT_DATA_DIR=f'{REPO_WORKSPACE}/{DATA_DIR}'
TRANSFORMED_WORKFLOW=f'{INPUT_DATA_DIR}/processed_nvt'
OUTPUT_DIR=f'{INPUT_DATA_DIR}/sessions_by_day'
MODEL_PATH = f'{INPUT_DATA_DIR}/saved_model'
# MODEL_PATH = './torch_ensemble/models'

print(f"INPUT_DATA_DIR: {INPUT_DATA_DIR}")
# print(f"TRANSFORMED_WORKFLOW: {TRANSFORMED_WORKFLOW}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")
print(f"MODEL_PATH: {MODEL_PATH}")

INPUT_DATA_DIR: workspace/data
OUTPUT_DIR: workspace/data/sessions_by_day
MODEL_PATH: workspace/data/saved_model


## Set the schema object

In [9]:
from merlin_standard_lib import Schema
# SCHEMA_PATH = os.environ.get("INPUT_SCHEMA_PATH", "/workspace/data/processed_nvt/schema.pbtxt")

SCHEMA_PATH = f'{TRANSFORMED_WORKFLOW}/schema.pbtxt'
schema = Schema().from_proto_text(SCHEMA_PATH)

In [10]:
import cloudpickle
loaded_model = cloudpickle.load(
                open(os.path.join(MODEL_PATH, "t4rec_model_class.pkl"), "rb")
            )

In [11]:
model = loaded_model.cuda()
model.eval()

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (to_merge): ModuleDict(
            (continuous_module): SequentialBlock(
              (0): ContinuousFeatures(
                (filter_features): FilterFeatures()
                (_aggregation): ConcatFeatures()
              )
              (1): SequentialBlock(
                (0): DenseBlock(
                  (0): Linear(in_features=2, out_features=64, bias=True)
                  (1): ReLU(inplace=True)
                )
              )
              (2): AsTabular()
            )
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (item_id-list): Embedding(491, 64, padding_idx=0)
                (category-list): Embedding(147, 64, padding_idx=0)
              )
            )
          )
          (_aggregation): ConcatFeatures()
        

## Trace the model


In [12]:
# train_paths = os.path.join(OUTPUT_DIR, f"{1}/train.parquet")
TRAIN_PATHS=f'{OUTPUT_DIR}/1/train.parquet'
dataset = Dataset(TRAIN_PATHS)

In [13]:
TRAIN_PATHS

'workspace/data/sessions_by_day/1/train.parquet'

In [14]:
sparse_max = {'age_days-list': 20,
 'weekday_sin-list': 20,
 'item_id-list': 20,
 'category-list': 20}

from transformers4rec.torch.utils.data_utils import MerlinDataLoader

def generate_dataloader(schema, dataset, batch_size=128, seq_length=20):
    loader = MerlinDataLoader.from_schema(
            schema,
            dataset,
            batch_size=batch_size,
            max_sequence_length=seq_length,
            shuffle=False,
            sparse_as_dense=True,
            sparse_max=sparse_max
        )
    return loader

In [15]:
loader = generate_dataloader(schema, dataset)
train_dict = next(iter(loader))

In [16]:
train_dict[0]['item_id-list']

tensor([[ 49,  36, 130,  ...,   0,   0,   0],
        [ 34,  16,  34,  ...,   0,   0,   0],
        [  3,   8,  14,  ...,   0,   0,   0],
        ...,
        [  4,  17,  95,  ...,   0,   0,   0],
        [ 23,   2,  23,  ...,   0,   0,   0],
        [174, 103,  25,  ...,   0,   0,   0]], device='cuda:0')

In [17]:
train_dict[0]

{'age_days-list': tensor([[0.4849, 0.8648, 0.1370,  ..., 0.0000, 0.0000, 0.0000],
         [0.9514, 0.9971, 0.5264,  ..., 0.0000, 0.0000, 0.0000],
         [0.5986, 0.7829, 0.8959,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.8389, 0.7179, 0.2957,  ..., 0.0000, 0.0000, 0.0000],
         [0.7645, 0.2106, 0.6182,  ..., 0.0000, 0.0000, 0.0000],
         [0.5982, 0.4947, 0.6335,  ..., 0.0000, 0.0000, 0.0000]],
        device='cuda:0'),
 'weekday_sin-list': tensor([[0.0140, 0.3720, 0.5400,  ..., 0.0000, 0.0000, 0.0000],
         [0.5489, 0.3369, 0.8028,  ..., 0.0000, 0.0000, 0.0000],
         [0.2827, 0.8099, 0.8618,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0313, 0.6036, 0.2137,  ..., 0.0000, 0.0000, 0.0000],
         [0.3610, 0.7434, 0.2133,  ..., 0.0000, 0.0000, 0.0000],
         [0.8821, 0.7026, 0.4364,  ..., 0.0000, 0.0000, 0.0000]],
        device='cuda:0'),
 'item_id-list': tensor([[ 49,  36, 130,  ...,   0,   0,   0],
         [ 34,  16,  34,  ...,   0,   

In [18]:
traced_model = torch.jit.trace(model, train_dict[0], strict=True)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [151]:
traced_model

Model(
  original_name=Model
  (heads): ModuleList(
    original_name=ModuleList
    (0): Head(
      original_name=Head
      (body): SequentialBlock(
        original_name=SequentialBlock
        (0): TabularSequenceFeatures(
          original_name=TabularSequenceFeatures
          (to_merge): ModuleDict(
            original_name=ModuleDict
            (continuous_module): SequentialBlock(
              original_name=SequentialBlock
              (0): ContinuousFeatures(
                original_name=ContinuousFeatures
                (filter_features): FilterFeatures(original_name=FilterFeatures)
                (_aggregation): ConcatFeatures(original_name=ConcatFeatures)
              )
              (1): SequentialBlock(
                original_name=SequentialBlock
                (0): DenseBlock(
                  original_name=DenseBlock
                  (0): Linear(original_name=Linear)
                  (1): ReLU(original_name=ReLU)
                )
              )
      

### Generate model input and output schemas to feed in the PredictPyTorch operator below.

In [20]:
input_schema = model.input_schema
output_schema = model.output_schema

In [21]:
input_schema

,name,tags,dtype,is_list,is_ragged,properties.int_domain.min,properties.int_domain.max
0,age_days-list,"(Tags.LIST, Tags.CONTINUOUS)",float32,True,False,0,0
1,weekday_sin-list,"(Tags.LIST, Tags.CONTINUOUS)",float32,True,False,0,0
2,item_id-list,"(Tags.ITEM_ID, Tags.ITEM, Tags.CATEGORICAL, Ta...",int64,True,False,0,490
3,category-list,"(Tags.CATEGORICAL, Tags.LIST)",int64,True,False,0,146


In [22]:
INPUT_DATA_DIR

'workspace/data'

In [118]:
import shutil
# ens_model_path = os.environ.get("ens_model_path", f"{INPUT_DATA_DIR}/models")

ens_model_path = f'workspace/data/models'

# Make sure we have a clean stats space for Dask
if os.path.isdir(ens_model_path):
    shutil.rmtree(ens_model_path)
os.mkdir(ens_model_path)

### We use `PredictPyTorch` operator that takes a pytorch model and packages it correctly for tritonserver to run on the PyTorch backend.

In [26]:
torch_op = input_schema.column_names >> PredictPyTorch(
    traced_model, input_schema, output_schema
)

In [27]:
ensemble = Ensemble(torch_op, input_schema)
ens_config, node_configs = ensemble.export(ens_model_path)

# Starting Triton Server

In [124]:
ens_model_path

'workspace/data/models'

In [120]:
# tritonserver --model-repository=/workspace/data/models

In [138]:
import tritonclient.http as client

# Create a triton client
try:
    triton_client = client.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))

client created.


In [139]:
# ensure triton is in a good state
triton_client.is_server_live()
triton_client.get_model_repository_index()

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>
POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '121'}>
bytearray(b'[{"name":"0_predictpytorchtriton","version":"1","state":"READY"},{"name":"ensemble_model","version":"1","state":"READY"}]')


[{'name': '0_predictpytorchtriton', 'version': '1', 'state': 'READY'},
 {'name': 'ensemble_model', 'version': '1', 'state': 'READY'}]

In [140]:
! curl -s -o /dev/null -w "%{http_code}" http://localhost:8000/v2/health/ready

200

In [141]:
eval_batch_size = 32
# eval_paths = os.path.join(OUTPUT_DIR, f"{1}/valid.parquet")
eval_paths=f'{OUTPUT_DIR}/1/valid.parquet'

eval_dataset = Dataset(eval_paths, shuffle=False)
eval_loader = generate_dataloader(schema, eval_dataset, batch_size=eval_batch_size)
test_dict = next(iter(eval_loader))

In [142]:
test_dict[0]['item_id-list'][0]

tensor([67,  7, 44,  4,  4, 83, 96,  9,  7, 99, 12, 16,  0,  0,  0,  0,  0,  0,
         0,  0], device='cuda:0')

In [143]:
# test_dict

In [144]:
df_cols = {}
for name, tensor in test_dict[0].items():
    if name in input_schema.column_names:
        df_cols[name] = tensor.cpu().numpy()
        if len(tensor.shape) > 1:
            df_cols[name] = list(df_cols[name])
            
df = make_df(df_cols)
print(df.shape)
df.head()

(32, 4)


,age_days-list,weekday_sin-list,item_id-list,category-list
0,"[0.5499501, 0.05859168, 0.6143423, 0.6127391, ...","[0.14183702, 0.033720933, 0.33820504, 0.261525...","[67, 7, 44, 4, 4, 83, 96, 9, 7, 99, 12, 16, 0,...","[15, 2, 9, 1, 1, 18, 20, 2, 2, 22, 2, 4, 0, 0,..."
1,"[0.054626163, 0.9909682, 0.06476167, 0.1239693...","[0.7294668, 0.19679245, 0.9772184, 0.018398901...","[7, 8, 77, 3, 11, 15, 8, 29, 67, 36, 0, 0, 0, ...","[2, 3, 16, 1, 2, 4, 3, 6, 15, 8, 0, 0, 0, 0, 0..."
2,"[0.932309, 0.80796856, 0.90713656, 0.15237969,...","[0.30309433, 0.32139105, 0.9710796, 0.2957704,...","[3, 17, 3, 11, 93, 152, 6, 54, 70, 54, 0, 0, 0...","[1, 4, 1, 2, 19, 32, 1, 11, 14, 11, 0, 0, 0, 0..."
3,"[0.54140306, 0.27389532, 0.71741486, 0.3560422...","[0.97362846, 0.36598137, 0.4796154, 0.5809341,...","[17, 98, 2, 32, 18, 22, 30, 2, 121, 22, 0, 0, ...","[4, 22, 1, 6, 4, 5, 6, 1, 25, 5, 0, 0, 0, 0, 0..."
4,"[0.12162325, 0.49388522, 0.31290212, 0.0548800...","[0.27499816, 0.40137193, 0.7278469, 0.2063758,...","[108, 1, 26, 121, 62, 35, 24, 15, 8, 0, 0, 0, ...","[23, 1, 7, 25, 13, 8, 5, 4, 3, 0, 0, 0, 0, 0, ..."


In [147]:
from merlin.systems.triton.utils import send_triton_request
response = send_triton_request(input_schema, df[input_schema.column_names], output_schema.column_names)
print(response)

{'next-item': array([[ -9.414455 ,  -3.3946676,  -3.5153584, ...,  -9.280288 ,
         -9.485153 , -10.177757 ],
       [ -9.41456  ,  -3.3946674,  -3.5153627, ...,  -9.280386 ,
         -9.48469  , -10.178482 ],
       [ -9.41453  ,  -3.394696 ,  -3.5153177, ...,  -9.280301 ,
         -9.485351 , -10.177533 ],
       ...,
       [ -9.414583 ,  -3.3947313,  -3.5153112, ...,  -9.280373 ,
         -9.485293 , -10.177692 ],
       [ -9.414576 ,  -3.39471  ,  -3.515358 , ...,  -9.280357 ,
         -9.485056 , -10.177971 ],
       [ -9.414605 ,  -3.394704 ,  -3.5153375, ...,  -9.280365 ,
         -9.485054 , -10.177982 ]], dtype=float32)}


In [156]:
response['next-item'].shape

(32, 491)

In [146]:
response['next-item'].shape

(32, 491)

In [ ]:
# REPO_WORKSPACE = 'workspace'

# DATA_DIR = 'data'
# INPUT_DATA_DIR=f'{REPO_WORKSPACE}/{DATA_DIR}'
# TRANSFORMED_WORKFLOW=f'{INPUT_DATA_DIR}/processed_nvt'
# OUTPUT_WORKFLOW_DIR=f'{INPUT_DATA_DIR}/workflow_etl'
# OUTPUT_DIR=f'{INPUT_DATA_DIR}/sessions_by_day'
# TRAIN_PATHS=f'{OUTPUT_DIR}/1' #/train.parquet'
# MODEL_PATH = f'{INPUT_DATA_DIR}/saved_model'

# print(f"INPUT_DATA_DIR: {INPUT_DATA_DIR}")
# print(f"TRANSFORMED_WORKFLOW: {TRANSFORMED_WORKFLOW}")
# print(f"OUTPUT_WORKFLOW_DIR: {OUTPUT_WORKFLOW_DIR}")
# print(f"OUTPUT_DIR: {OUTPUT_DIR}")
# print(f"TRAIN_PATHS: {TRAIN_PATHS}")
# print(f"MODEL_PATH: {MODEL_PATH}")

# Local Triton

In [41]:
LOCAL_MODEL_ARTIFACTS_DIR = "local_model_artifacts"

In [137]:
!pwd

/home/jupyter/t4rec-nvidia-docs


In [42]:
MODEL_NAME = "local_torch"
LOCAL_MODEL_DIRECTORY = f"{LOCAL_MODEL_ARTIFACTS_DIR}/{MODEL_NAME}"
LOCAL_MODEL_VERSION_1 = f"{LOCAL_MODEL_DIRECTORY}/1"

!mkdir $LOCAL_MODEL_ARTIFACTS_DIR
!mkdir $LOCAL_MODEL_DIRECTORY
!mkdir $LOCAL_MODEL_VERSION_1

In [43]:
LOCAL_MODEL_FILE = f"{LOCAL_MODEL_VERSION_1}/model.pt"

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
# traced_script_module = torch.jit.trace(model, example)

# Save the TorchScript model
traced_model.save(LOCAL_MODEL_FILE)

In [48]:
loaded_torch = torch.jit.load(LOCAL_MODEL_FILE)

def predict_class(instances):
    instances = torch.Tensor(instances)
    output = loaded_torch(instances)
    _, predicted = torch.max(output, 1)
    return predicted


# predicted = predict_class(test_dict[0])
# print([CLASS_VOCAB[class_index] for class_index in predicted])

In [49]:
predicted = loaded_torch(test_dict[0])

predicted

tensor([[ -9.4145,  -3.3947,  -3.5154,  ...,  -9.2803,  -9.4852, -10.1778],
        [ -9.4146,  -3.3947,  -3.5154,  ...,  -9.2804,  -9.4847, -10.1785],
        [ -9.4145,  -3.3947,  -3.5153,  ...,  -9.2803,  -9.4854, -10.1775],
        ...,
        [ -9.4146,  -3.3947,  -3.5153,  ...,  -9.2804,  -9.4853, -10.1777],
        [ -9.4146,  -3.3947,  -3.5154,  ...,  -9.2804,  -9.4851, -10.1780],
        [ -9.4146,  -3.3947,  -3.5153,  ...,  -9.2804,  -9.4851, -10.1780]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [135]:
test_dict

({'age_days-list': tensor([[0.5500, 0.0586, 0.6143, 0.6127, 0.5400, 0.5176, 0.9277, 0.0077, 0.1951,
           0.9919, 0.5984, 0.6831, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0546, 0.9910, 0.0648, 0.1240, 0.2173, 0.6687, 0.5286, 0.6902, 0.0695,
           0.3128, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.9323, 0.8080, 0.9071, 0.1524, 0.2586, 0.3491, 0.1346, 0.2808, 0.5879,
           0.1281, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.5414, 0.2739, 0.7174, 0.3560, 0.9111, 0.9626, 0.7585, 0.7505, 0.3491,
           0.6790, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1216, 0.4939, 0.3129, 0.0549, 0.2717, 0.6906, 0.6285, 0.0703, 0.6360,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.

In [50]:
%%writefile $LOCAL_MODEL_DIRECTORY/config.pbtxt
name: "local_t4r_torch"
platform: "pytorch_libtorch"
input {
  name: "age_days-list"
  data_type: TYPE_FP32
  dims: -1
  dims: -1
}
input {
  name: "weekday_sin-list"
  data_type: TYPE_FP32
  dims: -1
  dims: -1
}
input {
  name: "item_id-list"
  data_type: TYPE_INT64
  dims: -1
  dims: -1
}
input {
  name: "category-list"
  data_type: TYPE_INT64
  dims: -1
  dims: -1
}
output {
  name: "next-item"
  data_type: TYPE_FP32
  dims: -1
}
parameters {
  key: "INFERENCE_MODE"
  value {
    string_value: "true"
  }
}
backend: "pytorch"


Writing local_model_artifacts/local_torch/config.pbtxt


In [56]:
BUCKET_NAME='merlin-transformers4rec-jvt01'
BUCKET_URI=f'gs://{BUCKET_NAME}'

!gcloud storage cp -r $LOCAL_MODEL_ARTIFACTS_DIR/* $BUCKET_URI/local_model_dir/

Copying file://local_model_artifacts/local_torch/config.pbtxt to gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch/config.pbtxt
Copying file://local_model_artifacts/local_torch/1/model.pt to gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch/1/model.pt
  Completed files 2/2 | 795.2kiB/795.2kiB                                      

Average throughput: 75.9MiB/s


In [57]:
!gcloud storage ls $BUCKET_URI/local_model_dir/

gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch/


In [58]:
TRITON_VERSION = "23.01"
MODEL_NAME='local_t4r_torch'

In [110]:
from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel(
    serving_container_image_uri='gcr.io/hybrid-vertex/triton-ensemble_model-v02', #f"nvcr.io/nvidia/tritonserver:{TRITON_VERSION}-py3",
    # serving_container_predict_route=f"/v1/models/{MODEL_NAME}/infer",
    # serving_container_health_route=f"/v1/models/{MODEL_NAME}",
    serving_container_ports=[8000],
    serving_container_args=[
        "tritonserver", 
        "--model-repository=./workspace/data/models/", 
        "--backend-config=python,shm-default-byte-size=4194304",
        "--vertex-ai-default-model=0_predictpytorchtriton",
        "--strict-model-config=true",
    ],
    # serving_container_environment_variables=
)
# tritonserver --model-repository=./models/  --backend-config=python,shm-default-byte-size=4194304 

# ./workspace/data/models/
# "--model-repository=$(AIP_STORAGE_URI)",

In [111]:
local_model.get_serving_container_spec()

image_uri: "gcr.io/hybrid-vertex/triton-ensemble_model-v02"
args: "tritonserver"
args: "--model-repository=./workspace/data/models/"
args: "--backend-config=python,shm-default-byte-size=4194304"
args: "--vertex-ai-default-model=0_predictpytorchtriton"
args: "--strict-model-config=true"
ports {
  container_port: 8000
}

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>
POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '121'}>
bytearray(b'[{"name":"0_predictpytorchtriton","version":"1","state":"READY"},{"name":"ensemble_model","version":"1","state":"READY"}]')
[{'name': '0_predictpytorchtriton', 'version': '1', 'state': 'READY'},
 {'name': 'ensemble_model', 'version': '1', 'state': 'READY'}]

In [82]:
INPUT_FILE = "instances.json"

In [83]:
%%writefile $INPUT_FILE
{
    "id": "0",
    "inputs":[
        {
            "name": "age_days-list",
            "shape": [32,20],
            "datatype": "TYPE_FP32",
            "data":[
                [0.5500, 0.0586, 0.6143, 0.6127, 0.5400, 0.5176, 0.9277, 0.0077, 0.1951,
                0.9919, 0.5984, 0.6831, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.0546, 0.9910, 0.0648, 0.1240, 0.2173, 0.6687, 0.5286, 0.6902, 0.0695,
                0.3128, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.9323, 0.8080, 0.9071, 0.1524, 0.2586, 0.3491, 0.1346, 0.2808, 0.5879,
                0.1281, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.5414, 0.2739, 0.7174, 0.3560, 0.9111, 0.9626, 0.7585, 0.7505, 0.3491,
                0.6790, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.1216, 0.4939, 0.3129, 0.0549, 0.2717, 0.6906, 0.6285, 0.0703, 0.6360,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.3683, 0.6258, 0.7443, 0.5026, 0.5074, 0.9320, 0.3452, 0.9509, 0.4156,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.7699, 0.9812, 0.4180, 0.4125, 0.8800, 0.8031, 0.7665, 0.9540, 0.1402,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.1250, 0.5512, 0.0600, 0.3280, 0.3652, 0.0224, 0.3430, 0.0042, 0.0389,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.3052, 0.4312, 0.1452, 0.7214, 0.1445, 0.0432, 0.5242, 0.9162, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.7925, 0.4341, 0.9482, 0.2288, 0.2654, 0.3532, 0.1630, 0.2890, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.7368, 0.4399, 0.9752, 0.5455, 0.4462, 0.7319, 0.5730, 0.5451, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.2702, 0.7289, 0.6655, 0.4089, 0.8595, 0.1807, 0.4307, 0.7542, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.8401, 0.4632, 0.1454, 0.9137, 0.8883, 0.1295, 0.1184, 0.9455, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.7763, 0.0796, 0.9949, 0.6816, 0.7600, 0.7253, 0.2969, 0.1074, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.3500, 0.5600, 0.2751, 0.9803, 0.3926, 0.2887, 0.0542, 0.0389, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.0424, 0.4620, 0.1603, 0.8932, 0.3086, 0.8135, 0.7295, 0.5442, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.8814, 0.8263, 0.2925, 0.5446, 0.0098, 0.2152, 0.5280, 0.5267, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.8856, 0.5255, 0.6458, 0.4147, 0.4956, 0.1747, 0.3629, 0.1408, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.2657, 0.1204, 0.8135, 0.2827, 0.5359, 0.4308, 0.9232, 0.4348, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.2642, 0.0670, 0.0968, 0.2958, 0.9662, 0.1786, 0.5796, 0.5112, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.7902, 0.7060, 0.2042, 0.7567, 0.3623, 0.5933, 0.2972, 0.7807, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.2887, 0.8787, 0.0326, 0.1736, 0.8594, 0.2666, 0.1109, 0.0310, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.1795, 0.9626, 0.4311, 0.4249, 0.9850, 0.5115, 0.0170, 0.5917, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.2282, 0.7273, 0.4652, 0.9355, 0.7957, 0.6744, 0.3074, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.0340, 0.0742, 0.0409, 0.9161, 0.4526, 0.7397, 0.9096, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.5320, 0.1967, 0.2383, 0.0971, 0.7346, 0.8192, 0.9292, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.3977, 0.7373, 0.8947, 0.4379, 0.4440, 0.3884, 0.5187, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.9248, 0.4291, 0.6955, 0.5254, 0.9587, 0.0883, 0.8295, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.1657, 0.8969, 0.8151, 0.7413, 0.4698, 0.9391, 0.0023, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.1201, 0.0461, 0.9307, 0.5730, 0.3314, 0.5091, 0.0872, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.1567, 0.2906, 0.1418, 0.8887, 0.2787, 0.4564, 0.9123, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000],
                [0.7007, 0.6345, 0.4422, 0.2878, 0.1293, 0.3746, 0.2620, 0.0000, 0.0000,
                0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
                0.0000, 0.0000]
            ]
        },
        {
            "name": "weekday_sin-list",
            "shape": [32,20],
            "datatype": "TYPE_FP32",
            "data": [[
                1.4184e-01, 3.3721e-02, 3.3821e-01, 2.6153e-01, 6.4906e-01, 3.2920e-01,
                3.8131e-01, 3.6325e-01, 2.9700e-01, 6.3956e-01, 5.4634e-01, 9.7863e-01,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [7.2947e-01, 1.9679e-01, 9.7722e-01, 1.8399e-02, 2.2008e-01, 8.9536e-01,
                5.8914e-01, 8.0291e-01, 8.0971e-01, 5.4528e-01, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [3.0309e-01, 3.2139e-01, 9.7108e-01, 2.9577e-01, 1.6939e-01, 7.1141e-01,
                1.0686e-01, 4.8055e-01, 2.6228e-02, 3.7804e-01, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [9.7363e-01, 3.6598e-01, 4.7962e-01, 5.8093e-01, 1.9581e-01, 9.7743e-02,
                6.5561e-01, 9.5566e-01, 5.7583e-01, 6.4148e-01, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [2.7500e-01, 4.0137e-01, 7.2785e-01, 2.0638e-01, 4.8717e-01, 7.5873e-01,
                2.2807e-01, 6.7445e-01, 2.3984e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [5.1987e-01, 4.4924e-01, 4.0788e-02, 5.0338e-01, 7.5353e-01, 5.7805e-01,
                6.0641e-02, 9.8867e-01, 2.2639e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [5.6251e-02, 4.9211e-01, 7.2834e-01, 9.2414e-01, 3.7443e-01, 3.6146e-01,
                7.1508e-01, 5.6517e-03, 1.0645e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [1.6462e-01, 2.9790e-01, 1.6291e-01, 7.5653e-01, 1.1355e-01, 4.5202e-02,
                5.9896e-01, 1.8507e-01, 5.3856e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [3.9240e-01, 2.3877e-01, 8.1401e-01, 8.7395e-01, 2.0980e-01, 2.8778e-01,
                2.4794e-01, 6.0992e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [2.8330e-01, 4.5028e-01, 9.2351e-01, 7.4421e-01, 3.4624e-01, 1.4907e-02,
                8.7133e-01, 2.6281e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [4.4364e-02, 1.3227e-01, 9.1185e-01, 4.0432e-01, 4.1086e-02, 6.1182e-01,
                2.4097e-01, 2.7568e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [9.7054e-01, 9.4975e-01, 4.5498e-01, 3.0830e-03, 2.4583e-01, 7.7276e-01,
                2.2080e-01, 2.1478e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [1.5792e-01, 1.2624e-01, 3.2760e-01, 1.6827e-01, 5.1736e-01, 1.1740e-01,
                4.4619e-01, 7.4154e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [1.4120e-01, 9.7575e-01, 4.8281e-01, 3.9233e-01, 7.2846e-01, 2.0068e-01,
                1.7277e-01, 2.2205e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [2.1688e-01, 7.9391e-01, 3.4342e-01, 7.1375e-01, 4.4209e-02, 5.5700e-01,
                9.7416e-01, 7.2412e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [8.6845e-01, 9.2832e-01, 5.2192e-01, 1.0732e-01, 6.5297e-01, 2.5339e-01,
                9.6057e-01, 6.5247e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [9.8344e-03, 6.2103e-01, 4.8482e-01, 5.7600e-01, 6.3513e-01, 8.7940e-01,
                2.1914e-01, 7.6488e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [6.3450e-02, 8.4045e-02, 6.2344e-01, 3.5582e-01, 8.3470e-01, 1.5460e-01,
                5.4285e-01, 4.5440e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [8.5108e-01, 1.3929e-01, 1.8077e-01, 5.8014e-01, 9.4651e-01, 5.9727e-02,
                6.9440e-01, 9.4627e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [4.7142e-01, 9.0662e-01, 6.6303e-01, 4.4878e-01, 5.8265e-02, 9.0970e-01,
                3.1652e-01, 2.3523e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [3.0687e-01, 8.3835e-01, 7.0113e-01, 9.1762e-01, 3.8630e-01, 1.6809e-01,
                3.9336e-01, 1.5166e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [4.5343e-01, 9.0681e-01, 8.2771e-01, 6.9949e-01, 1.3863e-02, 6.0504e-01,
                7.5735e-01, 7.8476e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [2.0741e-02, 7.0131e-01, 2.9813e-01, 3.8618e-01, 3.5497e-01, 1.1390e-01,
                3.9853e-01, 8.1355e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [2.2238e-01, 1.8477e-01, 9.0214e-01, 4.4302e-01, 4.3727e-01, 6.8705e-01,
                4.5997e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [5.8187e-01, 6.7985e-01, 6.1613e-01, 7.9782e-01, 9.5726e-01, 3.1499e-01,
                9.9104e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [5.3236e-01, 6.0840e-01, 4.5391e-01, 2.8334e-01, 9.1774e-01, 3.3346e-01,
                7.4128e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [5.2920e-01, 2.0232e-01, 4.3059e-01, 8.9448e-01, 2.6901e-01, 1.4030e-01,
                8.5303e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [3.7203e-01, 6.2551e-01, 3.6959e-01, 5.8392e-01, 3.8439e-01, 4.4664e-01,
                4.0801e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [3.3538e-01, 6.0019e-01, 9.0847e-01, 2.8959e-01, 9.6694e-01, 5.9279e-01,
                4.5567e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [2.4335e-01, 9.0006e-01, 1.1879e-01, 1.6675e-01, 7.5352e-02, 1.4651e-01,
                7.7775e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [7.1672e-01, 2.0168e-01, 9.0585e-01, 8.1379e-01, 4.7558e-01, 2.1020e-01,
                3.9475e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00],
                [4.7117e-01, 2.0563e-01, 9.9711e-01, 3.4685e-01, 3.0795e-01, 9.8780e-01,
                1.6758e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
                0.0000e+00, 0.0000e+00]
            ]
        },
        {
            "name": "item_id-list",
            "shape": [32,20],
            "datatype": "TYPE_INT64",
            "data":[
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
                [67,7,44,4,4,83,96,9,7,99,12,16,0,0,0,0,0,0,0,0],
            ]
        },
        {
            "name": "category-list",
            "shape": [32,20],
            "datatype": "TYPE_INT64",
            "data":[
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
                [7,7,4,4,4,8,9,25,7,19,12,16,0,0,0,0,0,0,0,0],
            ]
        }
    ]
}

Writing instances.json


In [106]:
from google.oauth2 import service_account

CREDENTIALS_FILE = "credentials.json"
# t4rec-nvidia-docs/credentials.json
credentials = service_account.Credentials.from_service_account_file('credentials.json')

LOCAL_GCS_URI='gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch'

In [107]:
LOCAL_MODEL_ARTIFACTS_DIR

'local_model_artifacts'

In [108]:
from google.cloud import aiplatform as vertex_ai

vertex_ai.init(
    project=PROJECT_ID, 
    location=REGION,
    # credentials=credentials
)

In [113]:
# local_endpoint = local_model.deploy_to_local_endpoint(
#     artifact_uri=f"{LOCAL_MODEL_DIRECTORY}",
#     # credential_path=CREDENTIALS_FILE,
# )
# local_endpoint.serve()

In [ ]:
health_check_response = local_endpoint.run_health_check()
print(health_check_response, health_check_response.content)

In [ ]:
predict_response = local_endpoint.predict(
    request='{"instances": [[1, 2, 3, 4]]}',
    headers={"header-key": "header-value"},
)
print(predict_response, predict_response.content)

In [ ]:
local_endpoint.print_container_logs()

In [ ]:
# local_endpoint.stop()

In [114]:
# # pas local paths
# with local_model.deploy_to_local_endpoint(
#     artifact_uri=f"{LOCAL_MODEL_DIRECTORY}",
#     credential_path=CREDENTIALS_FILE,
# ) as local_endpoint:
#     health_check_response = local_endpoint.run_health_check()
#     print(health_check_response, health_check_response.content)
#     predict_response = local_endpoint.predict(
#         request_file=INPUT_FILE,
#         headers={"Content-Type": "application/json"},
#     )

    # health_check_response = local_endpoint.run_health_check()



# Pass GCP paths
# with local_model.deploy_to_local_endpoint(
#     artifact_uri=f"{LOCAL_GCS_URI}",
#     credential_path=CREDENTIALS_FILE,
# ) as local_endpoint:
#     predict_response = local_endpoint.predict(
#         request_file=INPUT_FILE,
#         headers={"Content-Type": "application/json"},
#     )

#     health_check_response = local_endpoint.run_health_check()

# >> DockerException: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))

In [115]:
# LOCAL_GCS_URI='gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch'

In [116]:
# INPUT_FILE